    '''
    https://docs.python.org/3/library/ast.html


    stmt = FunctionDef(identifier name, arguments args,
                       stmt* body, expr* decorator_list, expr? returns,
                       string? type_comment)
            | AsyncFunctionDef(identifier name, arguments args,
                             stmt* body, expr* decorator_list, expr? returns,
                             string? type_comment)

            | ClassDef(identifier name,
             expr* bases,
             keyword* keywords,
             stmt* body,
             expr* decorator_list)
            | Return(expr? value)
            | Delete(expr* targets)
            | Assign(expr* targets, expr value, string? type_comment)
            | AugAssign(expr target, operator op, expr value)
            -- 'simple' indicates that we annotate simple name without parens
            | AnnAssign(expr target, expr annotation, expr? value, int simple)

            -- use 'orelse' because else is a keyword in target languages
            | For(expr target, expr iter, stmt* body, stmt* orelse, string? type_comment)
            | AsyncFor(expr target, expr iter, stmt* body, stmt* orelse, string? type_comment)
            | While(expr test, stmt* body, stmt* orelse)
            | If(expr test, stmt* body, stmt* orelse)
            | With(withitem* items, stmt* body, string? type_comment)
            | AsyncWith(withitem* items, stmt* body, string? type_comment)

            | Match(expr subject, match_case* cases)

            | Raise(expr? exc, expr? cause)
            | Try(stmt* body, excepthandler* handlers, stmt* orelse, stmt* finalbody)
            | Assert(expr test, expr? msg)

            | Import(alias* names)
            | ImportFrom(identifier? module, alias* names, int? level)

            | Global(identifier* names)
            | Nonlocal(identifier* names)
            | Expr(expr value)
            | Pass | Break | Continue

    expr = BoolOp(boolop op, expr* values)
            | NamedExpr(expr target, expr value)
            | BinOp(expr left, operator op, expr right)
            | UnaryOp(unaryop op, expr operand)
            | Lambda(arguments args, expr body)
            | IfExp(expr test, expr body, expr orelse)
            | Dict(expr* keys, expr* values)
            | Set(expr* elts)
            | ListComp(expr elt, comprehension* generators)
            | SetComp(expr elt, comprehension* generators)
            | DictComp(expr key, expr value, comprehension* generators)
            | GeneratorExp(expr elt, comprehension* generators)
            -- the grammar constrains where yield expressions can occur
            | Await(expr value)
            | Yield(expr? value)
            | YieldFrom(expr value)
            -- need sequences for compare to distinguish between
            -- x < 4 < 3 and (x < 4) < 3
            | Compare(expr left, cmpop* ops, expr* comparators)
            | Call(expr func, expr* args, keyword* keywords)
            | FormattedValue(expr value, int conversion, expr? format_spec)
            | JoinedStr(expr* values)
            | Constant(constant value, string? kind)

            -- the following expression can appear in assignment context
            | Attribute(expr value, identifier attr, expr_context ctx)
            | Subscript(expr value, expr slice, expr_context ctx)
            | Starred(expr value, expr_context ctx)
            | Name(identifier id, expr_context ctx)
            | List(expr* elts, expr_context ctx)
            | Tuple(expr* elts, expr_context ctx)

            -- can appear only in Subscript
            | Slice(expr? lower, expr? upper, expr? step)

    '''

In [1]:
import sys
import ast
import json
from ast2json import ast2json


In [2]:

# sanitize input...
filename: str = "../test-cases/test1.py"

ast_tree = ast.parse(open(filename).read())
json_tree = ast2json(ast_tree)
print(json.dumps(json_tree, indent=4))


{
    "_type": "Module",
    "body": [
        {
            "_type": "FunctionDef",
            "args": {
                "_type": "arguments",
                "args": [
                    {
                        "_type": "arg",
                        "annotation": {
                            "_type": "Name",
                            "col_offset": 17,
                            "ctx": {
                                "_type": "Load"
                            },
                            "end_col_offset": 21,
                            "end_lineno": 4,
                            "id": "bool",
                            "lineno": 4
                        },
                        "arg": "a",
                        "col_offset": 14,
                        "end_col_offset": 21,
                        "end_lineno": 4,
                        "lineno": 4,
                        "type_comment": null
                    },
                    {
                        

In [3]:
supported_type_map: dict = {
    'int': int,
    'bool': bool,
    'float': float,
    'str': str,
    'chr': chr,
}


In [4]:
stmt_type_map: dict = {
    'If': '',
}

test_type_map: dict = {
    'BoolOp': '',
    'Name': '',

}

arg_type_map: dict = {
    "bool": [True, False],
    "int": range(-sys.maxsize, sys.maxsize),
    "float": 2 ** 64,
}


In [5]:
def detect_branching(line) -> bool:
    # print(line)

    line_type = line['_type']
    # if line_type in stmt_type_map:
    #     print(line)

    if line_type == 'If':

        test = line['test']
        test_type = test['_type']

        if test_type == 'BoolOp':
            op_type = test['op']['_type']
            print(op_type)
            for value in test['values']:
                continue
                #print(value)
                # var_id = value['id']

        if test_type == 'Name':
            test_var_id = test['id']
            #print(test_var_id)

        #print(test)
        return True

    return False


In [6]:
for body in json_tree['body']:
    if body["_type"] != "FunctionDef":
        continue  # guard clause

    func_name = body['name']
    print(f"FUNCTION: {func_name}")

    branch_count: int = 1

    args = body['args']['args']
    # function args

    states = 1
    for arg in args:
        # print(arg)
        arg_type = arg['annotation']['id']

        if arg_type in arg_type_map:
            arg_val_range = arg_type_map[arg_type]
            states = states * len(arg_val_range)
        arg_identifier = arg['arg']
    print(f"input states size: {states}")

    for line in body['body']:
        if detect_branching(line):
            branch_count += 1

    print(f"Branch Count: {branch_count}")


FUNCTION: test_func
input states size: 4
And
And
Branch Count: 4
FUNCTION: test_func2
input states size: 2
And
And
Branch Count: 4
